In [ ]:

#################################################### 1. monocle3 infer snRNA RGCs trajectory #######################################################
# 1. R import packages
rm(list = ls())
library(monocle3)
library(ggplot2)
library(reticulate)
library(dplyr)
library(cowplot)
library(ggpubr)
library(pheatmap)
library(Matrix)
library(Seurat)
library(RColorBrewer)
defaultW <- getOption("warn")
options(warn = -1)

# 2. define variables
bin_type = "cellbin"
setwd("/data/work/human_embryo")
sample <- "neural_cells"
domain_key <- "cell_subgroup"

# 3. define paths
## snRNA
rds_path <- paste0("snRNA/snRNA.brain")
rds.temp <- readRDS(paste0(rds_path, ".rds"))
Idents(rds.temp) <- domain_key
DimPlot(rds.temp, label=T, split.by = domain_key)

# 4. monocle3
dir.create("snRNA/monocle3")
final_path <- paste0("snRNA/monocle3/", sample, ".")
# 1.load data to monocle3
cds = Seurat2Monocle3(rds.temp)
class(cds)

cds <- preprocess_cds(cds, num_dim = 50)
cds@int_colData$reducedDims$PCA <- rds.temp@reductions$pca@cell.embeddings
# cds@int_colData$reducedDims$UMAP <- rds.temp@reductions$umap@cell.embeddings
cds <- reduce_dimension(cds, reduction_method="UMAP")

plot_tmp <- plot_pc_variance_explained(cds)
ggsave(filename = paste0("snRNA/monocle3/", sample, ".pc_variance_explained.pdf"),
       plot =  plot_tmp,
       # width=15, #width=8.9,
       # height=30,
       units = "cm",
       dpi=300 ) 
# cds <- align_cds(cds, num_dim = 100, alignment_group = "sample")  # get_citations()


print(head(fData(cds)))
print(head(pData(cds)))
pData(cds)[[domain_key]] <- as.character(pData(cds)[[domain_key]])
saveRDS(cds, file = paste0(final_path, "cds.rds"))

pdf(paste0(final_path, domain_key, ".pdf"))
plot_cells(cds, color_cells_by="stage")
plot_cells(cds, color_cells_by="celltype")
plot_cells(cds, color_cells_by=domain_key) #+ ggtitle('cds.umap')
dev.off()
        
# 5.Learn the trajectory graph
cds <- learn_graph(cds)
saveRDS(cds, file = paste0(sample, ".cds.rds"))
##plot
pdf(paste0(final_path, "clusters", ".graph.pdf"))
plot_cells(cds) + ggtitle(paste0("clusters"))
plot_cells(cds, color_cells_by="partition", group_cells_by="partition") + ggtitle(paste0("partition")) #labels_per_group=2
plot_cells(cds, color_cells_by="stage", label_groups_by_cluster=FALSE) + ggtitle(paste0("stage"))
plot_cells(cds, color_cells_by=domain_key, label_groups_by_cluster=FALSE) + ggtitle(paste0(domain_key))
plot_cells(cds, color_cells_by = domain_key, label_cell_groups=FALSE, label_leaves=TRUE, label_branch_points=TRUE, graph_label_size=1.5) + ggtitle(paste0(domain_key))             
dev.off()

# 6. automatically select root and order cells
get_earliest_principal_node <- function(cds, time_bin="CS19"){
  cell_ids <- which(colData(cds)[, "stage"] == time_bin)
  closest_vertex <- cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <- igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names(which.max(table(closest_vertex[cell_ids,]))))]
  root_pr_nodes
}
cds <- order_cells(cds, root_pr_nodes=get_earliest_principal_node(cds))
saveRDS(cds, file = paste0(final_path, "cds.root.rds"))
pdf(paste0(final_path, "pseudotime", ".pdf"))
plot_cells(cds, color_cells_by = "pseudotime", label_cell_groups=FALSE, label_leaves=FALSE, label_branch_points=FALSE, graph_label_size=1.5, show_trajectory_graph = TRUE)
dev.off()

# 7. identify track genes and plot
cds = readRDS(paste0(final_path, "cds.root.rds"))
track_genes <- graph_test_new(cds, neighbor_graph="principal_graph", cores=15) # knn principal_graph
row.names(subset(track_genes, q_value < 0.05))
write.table(data.frame("genes"=rownames(track_genes), track_genes), paste0(final_path, 'track_gene.tab'), sep='\t', quote=T, row.names=F, col.names=T)

features <- c("NEUROG2", "FOXG1", "HMGA2", "HES5", 'FOXP2', 'NR2F2', 'FYN', 'NFIB', 'EOMES', 'LHX2', 'CHD7', 'EMX2', 'FEZF2', 'SOX3', 'LEF1', 'HES1', 'GLI3', 'SOX2', 'IGF2BP1', 'SALL1', 'FAT4', 'PAX6', 'FOXG1', 'NR2E1', 'DMRTA2', 'ID4', 'TCF4', 'BCL11A', 'SOX9', 'NLGN1', 'SFRP2', 'BRINP3', 'CDON', 'SOX5', 'TENM4', 'SEMA6D', 'PLAG1', 'IL1RAPL1', 'YAP1') 
pdf(paste0(final_path, "genes", ".pseudo.cds.pdf"), width=4, height=4)
for(feature in features){
    p1 <- plot_genes_in_pseudotime(cds[c(feature),], cell_size = 0, ncol=1) + scale_y_continuous(limits = c(0,4), breaks = seq(0,4,2)) #+ scale_color_manual(values=colorset)
    print(p1)
}
dev.off()     
        
Seurat2Monocle3=function(sce, do.init=T){
  # get data
  #expr_matrix=as.matrix(sce@assays$RNA@counts)
  expr_matrix=GetAssayData(sce, assay = 'RNA', slot = 'counts')
  cell_meta=sce@meta.data
  gene_annotation=data.frame(
    gene_short_name=row.names(sce), #must have this column
    row.names = row.names(sce)
  )
  # new obj
  cds <- new_cell_data_set(expr_matrix,
                         cell_metadata = cell_meta,
                         gene_metadata = gene_annotation)

  return(cds)
}

In [ ]:

#################################################### 2. calculate bin50 pathway score #######################################################

library(Seurat)
library(UCell)
library(irGSEA)
library(RColorBrewer)
library(circlize)
library(reshape2)
library(ggplot2)
library(plyr)
library(dplyr)
library(viridis)
setwd(paste0("/data/work/human_embryo/"))
bin_type = "bin50"

sample_list <- c("CS12-13 E1S1")
for(sample_temp in sample_list){
    rds_path <- paste0("rds/", sample_temp, ".bin50")
    if(file.exists(paste0(rds_path, ".rds")) ){
        print(paste0(sample_temp, " exist!"))
        mydata <- readRDS(paste0(rds_path, ".rds"))
        mydata <- irGSEA.score(object = mydata, assay = "RNA",
                               slot = "data", seeds = 123, ncores = 10,
                               min.cells = 3, min.feature = 0,
                               msigdb = T,
                               species = "Homo sapiens", 
                               category = "C5",
                               subcategory = "BP",
                               geneid = "symbol",
                               minGSSize=8, # maxGSSize=1000,
                               method = c("UCell"),
                               aucell.MaxRank = NULL, ucell.MaxRank = NULL,
                               kcdf = 'Gaussian'
                              )
        saveRDS(mydata, paste0(rds_path, ".anno.rds"))
        for(assay in c("UCell")){
            scores_table <- as.matrix(GetAssayData(mydata, assay = assay, slot = "data"))
            write.table(data.frame("cell"=rownames(t(scores_table)), t(scores_table)), file=paste0(rds_path, ".", assay, ".tab"), sep="\t", row.names=F)
         }
    }
    else{
        print(paste0(sample_temp, " not exist!"))
    }
}

